In [1]:
import numpy as np

In [2]:
class CurrentBoard:
    def __init__(self,setup_of_board = "         "):
        self.X = "X"
        self.O = "O"
        self.space = " "
        
        if len(setup_of_board) !=9:
            print("Board has to have 9 entries")
        else:
            self.board = setup_of_board
            self.outcome = self.game_won()
            self.game_board = self.construct_display_string()
        
    def construct_display_string(self):
        hold = ""
        for i in range(len(self.board)):
            if self.board[i] == self.space:
                hold += str(i)
            else:
                hold += self.board[i]
        return hold
            
    def display_gameboard(self):
        self.display(self.game_board)
    
    def display(self, board_string= ""):
        if board_string == "":
            c = self.board
        else:
            c = board_string
            
        print("{}|{}|{}".format(c[0],c[1],c[2]))
        print("-----")
        print("{}|{}|{}".format(c[3],c[4],c[5]))
        print("-----")
        print("{}|{}|{}".format(c[6],c[7],c[8]))
        
    
    def eq3(self, i,j,k):
        return (self.board[i] == self.board[j]) and (self.board[j] == self.board[k]) and (self.board[i] != self.space)
    
    def game_won(self):
        # Return X if X wins, O if O wins and " " if a draw or not over
        if self.eq3(0,1,2) or self.eq3(0,4,8) or self.eq3(0,3,6):
            return self.board[0]
        if self.eq3(1,4,7) or self.eq3(3,4,5) or self.eq3(2,4,6):
            return self.board[4]
        if self.eq3(2,5,8) or self.eq3(6,7,8):
            return self.board[8]      
        return self.space
    
    
    def Other(self,character):
        if character == self.X:
            return self.O
        if character == self.O:
            return self.X
        print("Error Character not recognised")
        return self.space
        
        
    def all_possible_moves_for(self, choosing_this_character):
        
        list_of_all_possible_moves = []
        for index in range(len(self.board)):
            if self.board[index] == self.space:
                list_of_all_possible_moves.append( CurrentBoard(self.board[:index]+choosing_this_character + self.board[index+1:]))
            
        
        
        
        return list_of_all_possible_moves
    

In [3]:
game = CurrentBoard(" X O  X  ")

In [4]:
game.display_gameboard()

0|X|2
-----
O|4|5
-----
X|7|8


In [5]:
class SearchTreeNode:
    def __init__(self, just_after_choosing_character, Current_Game = None, Ply_Depth = 0):
        self.children = []
        self.ply_depth = Ply_Depth
        self.is_node_value_assigned = False
        self.node_value = 0
        self.choosing_character = just_after_choosing_character
        if Current_Game == None:
            self.game_at_this_node = CurrentBoard()
        else: 
            self.game_at_this_node = Current_Game
            
        self.generate_children()
        
    def generate_children(self):
        if self.game_at_this_node.game_won() != self.game_at_this_node.space:
            #Leaf apply utility function
            if(self.ply_depth % 2) == 1:
                self.node_value = 1
            else:
                self.node_value = -1
            self.is_node_value_assigned = True
        else:
            all_moves = self.game_at_this_node.all_possible_moves_for(self.game_at_this_node.Other(self.choosing_character))
            if (len(all_moves)==0):
                     self.node_value = 0
                     self.is_node_value_assigned = True
            else:    
                for new_game in all_moves:
                    self.children.append(SearchTreeNode(new_game.Other(self.choosing_character),new_game, self.ply_depth+1))

    def get_value(self):
        if not self.is_node_value_assigned:
   
            self.is_node_value_assigned = True
            self.children = sorted(self.children, key = lambda x: x.get_value()) 
            
            if (self.ply_depth % 2) == 0: #Maximising Layer
                self.node_value = self.children[-1].node_value              
            else: #Minimising Layer
                self.node_value = self.children[0].node_value            
                
        return self.node_value
        

In [7]:
G = SearchTreeNode("O",CurrentBoard("XOO    X "))

In [8]:
for c in G.children[0].children[0].children[0].children:
    print ("This child has value " + str(c.node_value))
    c.game_at_this_node.display()

This child has value -1
X|O|O
-----
X|O|X
-----
O|X| 
This child has value 0
X|O|O
-----
X|O|X
-----
 |X|O


In [9]:
G.game_at_this_node.display()

X|O|O
-----
 | | 
-----
 |X| 


In [10]:
G.get_value()

1

In [11]:
G.children

In [12]:
G.children[-1].game_at_this_node.display()

X|O|O
-----
 | | 
-----
 |X|X


In [13]:
G.children[3].game_at_this_node. display()

X|O|O
-----
 | | 
-----
X|X| 


In [14]:
all_moves = G.game_at_this_node.all_possible_moves_for("O")

In [15]:
for b in all_moves:
    print()
    b.display()
    print()



X|O|O
-----
O| | 
-----
 |X| 


X|O|O
-----
 |O| 
-----
 |X| 


X|O|O
-----
 | |O
-----
 |X| 


X|O|O
-----
 | | 
-----
O|X| 


X|O|O
-----
 | | 
-----
 |X|O



In [25]:
cb = CurrentBoard("OX   O XX")

In [26]:
cb.display()

O|X| 
-----
 | |O
-----
 |X|X


In [27]:
all_moves = cb.all_possible_moves_for("O")

In [29]:
for b in all_moves:
    print()
    b.display()


O|X|O
-----
 | |O
-----
 |X|X

O|X| 
-----
O| |O
-----
 |X|X

O|X| 
-----
 |O|O
-----
 |X|X

O|X| 
-----
 | |O
-----
O|X|X


In [30]:
def play_tic_tac_toe():
    current_board = None
    player_choosing = ""
    computer_choosing = ""

    choice = input("Do your want to go first? y/n")
    player_going_first = (choice == "y")
    choice = input("Do you want to play X? y/n")
    
    if choice == "y":
        player_choosing = "X"
        computer_choosing = "O"
    else:
        player_choosing = "O"
        computer_choosing = "X"
        
    if player_going_first:
        current_board = CurrentBoard("         ")
    else:
        # choose a random move
        empty = "         "
        index = np.random.randint(len(empty))
        current_board = CurrentBoard(empty[:index] + computer_choosing + empty[index+1:])
        
    for i in range(1000):
        current_board.display_gameboard()
#         current_board.display()
        index_as_string = input("Choose your move ")
        index = int(index_as_string)
        c_b_string = current_board.board
        current_board = CurrentBoard(c_b_string[:index] + player_choosing + c_b_string[index+1:])

        if current_board.outcome != current_board.space:
            print("I dont believe it!! you won")
            break
        elif len(current_board.all_possible_moves_for(computer_choosing)) == 0:
            print("Congrats you drew!!!")
            break

        search_tree = SearchTreeNode(player_choosing, current_board)
        search_tree.get_value()
        current_board = search_tree.children[-1].game_at_this_node

        if current_board.outcome != current_board.space:
            print("You are lost")
            break
        elif len(current_board.all_possible_moves_for(player_choosing)) == 0:
            print("Congrats you drew!!!")
            break

    
    return player_going_first, player_choosing, current_board, search_tree

In [31]:
a, b, c, s = play_tic_tac_toe()

Do your want to go first? y/ny
Do you want to play X? y/ny
0|1|2
-----
3|4|5
-----
6|7|8
Choose your move 0
X|1|2
-----
3|O|5
-----
6|7|8
Choose your move 8
X|1|2
-----
3|O|5
-----
6|O|X
Choose your move 1
X|X|O
-----
3|O|5
-----
6|O|X
Choose your move 6
X|X|O
-----
O|O|5
-----
X|O|X
Choose your move 5
Congrats you drew!!!


In [32]:
s.children[-1].game_at_this_node.display()

X|X|O
-----
O|O| 
-----
X|O|X


In [33]:
c.display_gameboard()

X|X|O
-----
O|O|X
-----
X|O|X
